TODO: clean up and add more notes in later sections

In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio

pio.templates["custom"] = pio.templates["plotly"]
pio.templates["custom"]["layout"]["colorway"] = px.colors.sequential.RdBu
pio.templates.default = "custom"


from matplotlib.pyplot import ylabel

In [ ]:
results = torch.load('output.pt')
plt.plot(results['train_losses'])
plt.plot(results['test_losses'])

plt.yscale('log')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss'])

In [ ]:
results.keys()

In [ ]:
results = torch.load('output_plus_and_minus.pt', map_location=torch.device('cpu'))
plt.plot(results['train_losses'])
plt.plot(results['test_losses'])

plt.yscale('log')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train_loss', 'test_loss',])


In [ ]:
results.keys()
plt.plot(results['train_precision_scores'])
plt.plot(results['test_precision_scores'])
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['train', 'test'])


In [ ]:
for operation in results['operations_losses']:
    plt.plot(operation['train_losses'])
for operation in results['operations_losses']:
    plt.plot(operation['test_losses'])
plt.yscale('log')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['+ train', '- train', '+ test', '- test'])


In [ ]:
for operation in results['operations_losses']:
    plt.plot(operation['train_precisions'])
for operation in results['operations_losses']:
    plt.plot(operation['test_precisions'])
plt.ylabel('precision')
plt.xlabel('epoch')
plt.legend(['+ train', '- train', '+ test', '- test'])
plt.yscale('log')

In [ ]:
from transformer_lens import HookedTransformer, HookedTransformerConfig, HookedEncoderDecoder

In [ ]:
results['config'].device = 'cpu'

In [ ]:
model = HookedTransformer(results['config'])
model.load_state_dict(results['model'])

## Show model works

In [ ]:
# (1 + 15) % 113
model(torch.tensor([1, 114, 15, 113]))[0,3].argmax().item()

In [ ]:
# (1 - 15) % 113
model(torch.tensor([1, 115, 15, 113]))[0, 3].argmax().item()

## Run model on full dataset to look at activations/attention patterns

In [ ]:
import einops
from modular_addition import ModularOperationsDataset
operations = (lambda x, y: x + y, lambda x, y: x - y)
dataset = ModularOperationsDataset(
    base=113,
    train_fraction=0.25,
    operations=operations,
)
full_dataset = einops.rearrange(dataset.data, "i j k -> (i k) j")
plus_dataset = dataset.data[:,:,0]
minus_dataset = dataset.data[:,:,1]
print(f"plus: {plus_dataset.shape}, minus: {minus_dataset.shape}, full: {full_dataset.shape}")

In [ ]:
output, cache = model.run_with_cache(full_dataset)

In [ ]:
av_attention = cache["pattern", 0].mean(dim=0).detach().cpu()

In [ ]:
labels = ['a', 'operation', 'b', '=']
# Create a figure to hold the 4 attention head plots
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("Attention Patterns for 4 Heads", fontsize=16)

# Loop through each head and plot its attention pattern
for i, ax in enumerate(axes):
    sns.heatmap(
        av_attention[i],
        annot=True,            # Annotate each cell with its value
        xticklabels=labels,    # Set x-axis labels
        yticklabels=labels,    # Set y-axis labels
        cmap="viridis",        # Use a color map
        cbar=False,            # Disable color bar to reduce clutter
        ax=ax,                  # Plot on the current axis
    )
    ax.set_title(f'Head {i + 1}', fontsize=12)
    ax.set_ylabel('destination token', fontsize=12)
    ax.set_xlabel('source token', fontsize=12)

# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

In [ ]:
# plot just attn where = is the destination (all we care about)
labels = ['a', 'operation', 'b', '=']
# Create a figure to hold the 4 attention head plots
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("Attention Patterns for 4 Heads", fontsize=16)

# Loop through each head and plot its attention pattern
for i, ax in enumerate(axes):
    sns.heatmap(
        av_attention[i, -1:],
        annot=True,            # Annotate each cell with its value
        xticklabels=labels,    # Set x-axis labels
        yticklabels=["="],    # Set y-axis labels
        cmap="viridis",        # Use a color map
        cbar=False,            # Disable color bar to reduce clutter
        ax=ax                  # Plot on the current axis
    )
    ax.set_title(f'Head {i + 1}', fontsize=12)
    ax.set_ylabel('destination token', fontsize=12)
    ax.set_xlabel('source token', fontsize=12)

# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

### Attention patterns for +

In [ ]:
output_plus, cache_plus = model.run_with_cache(plus_dataset)
av_attention = cache_plus["pattern", 0].mean(dim=0).detach().cpu()
labels = ['a', 'operation', 'b', '=']
# Create a figure to hold the 4 attention head plots
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("Attention Patterns for 4 Heads", fontsize=16)

# Loop through each head and plot its attention pattern
for i, ax in enumerate(axes):
    sns.heatmap(
        av_attention[i],
        annot=True,  # Annotate each cell with its value
        xticklabels=labels,  # Set x-axis labels
        yticklabels=labels,  # Set y-axis labels
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'Head {i + 1}', fontsize=12)

# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

### Attention patterns for -

In [ ]:
output_minus, cache_minus = model.run_with_cache(minus_dataset)
av_attention = cache_minus["pattern", 0].mean(dim=0).detach().cpu()
labels = ['a', 'operation', 'b', '=']
# Create a figure to hold the 4 attention head plots
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("Attention Patterns for 4 Heads", fontsize=16)

# Loop through each head and plot its attention pattern
for i, ax in enumerate(axes):
    sns.heatmap(
        av_attention[i],
        annot=True,  # Annotate each cell with its value
        xticklabels=labels,  # Set x-axis labels
        yticklabels=labels,  # Set y-axis labels
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'Head {i + 1}', fontsize=12)

# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

Next we plot the attention patterns for all combinations of a and b. Here we expect this to be very similar to what was found in the original paper with some potential interesting stuff on the openeration as the source attention.

In [ ]:
#Original model form paper
import einops
original_results = torch.load('output.pt')
original_model = HookedTransformer(original_results['config'])
original_model.load_state_dict(original_results['model'])
a_s = einops.repeat(torch.arange(113), "i -> (i j)", j=113)
b_s = einops.repeat(torch.arange(113), "j -> (i j)", i=113)
equals = einops.repeat(
    torch.tensor(113), " -> (i j)", i=113, j=113
)
original_dataset_correctly_ordered = torch.stack([a_s, b_s, equals], dim=1)
original_output, original_cache = original_model.run_with_cache(original_dataset_correctly_ordered)
a_to_equals_attn_patterns = original_cache["pattern", 0][:,:,-1,0].detach().cpu()
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
fig.suptitle("attention Patterns for a -> = for original model", fontsize=16)

# Loop through each head and plot its attention pattern

for i, ax in enumerate(axes):
    sns.heatmap(
        a_to_equals_attn_patterns[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'Head {i + 1}', fontsize=12)
# Loop through each head and plot its attention pattern

# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

In [ ]:
a_to_equals_attn_patterns_plus = cache_plus["pattern", 0][:,:,-1,0].detach().cpu()
a_to_equals_attn_patterns_minus = cache_minus["pattern", 0][:,:,-1,0].detach().cpu()
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Attention Patterns a -> =", fontsize=16)

# Loop through each head and plot its attention pattern

for i, ax in enumerate(axes[0]):
    sns.heatmap(
        a_to_equals_attn_patterns_plus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'+: Head {i + 1}', fontsize=12)

for i, ax in enumerate(axes[1]):

    sns.heatmap(
        a_to_equals_attn_patterns_minus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'-: Head {i + 1}', fontsize=12)
# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

In [ ]:
operation_to_equals_attn_patterns_plus = cache_plus["pattern", 0][:,:,-1,1].detach().cpu()
operation_to_equals_attn_patterns_minus = cache_minus["pattern", 0][:,:,-1,1].detach().cpu()
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Attention Patterns operation -> =", fontsize=16)

# Loop through each head and plot its attention pattern

for i, ax in enumerate(axes[0]):
    sns.heatmap(
        operation_to_equals_attn_patterns_plus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'+: Head {i + 1}', fontsize=12)

for i, ax in enumerate(axes[1]):

    sns.heatmap(
        operation_to_equals_attn_patterns_minus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'-: Head {i + 1}', fontsize=12)
# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

In [ ]:
b_to_equals_attn_patterns_plus = cache_plus["pattern", 0][:,:,-1,2].detach().cpu()
b_to_equals_attn_patterns_minus = cache_minus["pattern", 0][:,:,-1,2].detach().cpu()
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Attention Patterns b -> =", fontsize=16)

# Loop through each head and plot its attention pattern

for i, ax in enumerate(axes[0]):
    sns.heatmap(
        b_to_equals_attn_patterns_plus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'+: Head {i + 1}', fontsize=12)

for i, ax in enumerate(axes[1]):

    sns.heatmap(
        b_to_equals_attn_patterns_minus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'-: Head {i + 1}', fontsize=12)
# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

In [ ]:
equals_to_equals_attn_patterns_plus = cache_plus["pattern", 0][:,:,-1,3].detach().cpu()
equals_to_equals_attn_patterns_minus = cache_minus["pattern", 0][:,:,-1,3].detach().cpu()
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
fig.suptitle("Attention Patterns = -> =", fontsize=16)

# Loop through each head and plot its attention pattern

for i, ax in enumerate(axes[0]):
    sns.heatmap(
        equals_to_equals_attn_patterns_plus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'+: Head {i + 1}', fontsize=12)

for i, ax in enumerate(axes[1]):

    sns.heatmap(
        equals_to_equals_attn_patterns_minus[:, i].reshape(113, 113),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
        ax=ax  # Plot on the current axis
    )
    ax.set_title(f'-: Head {i + 1}', fontsize=12)
# Show the plot
plt.tight_layout()
plt.subplots_adjust(top=0.85)  # Adjust to make space for the title
plt.show()

Definite periodicity but some interesting stuff happening. Maybe the model is learning a similar alg to the original but with more frequency components to accurately calculate both operations

## Check the embedding matrix

In [ ]:
W_E = model.embed.W_E.cpu().detach()[:113]
sns.heatmap(
        W_E.numpy(),
        cmap="viridis",  # Use a color map
        cbar=False,  # Disable color bar to reduce clutter
    )

In [ ]:
U, S, Vh = torch.svd(W_E)
plt.plot(S)
plt.title('W_E singular values')

In [ ]:


# Create heatmap
px.imshow(U)

In [ ]:
px.line(U[:, :8].T, title="Principle Components of Embedding").update_layout(
    xaxis_title="Input Vocabulary")

In [ ]:
fourier_basis = []
fourier_basis_names = []
fourier_basis.append(torch.ones(113))
fourier_basis_names.append('Constant')
for freq in range(1, 113//2 + 1):
    fourier_basis.append(torch.sin(torch.arange(113)*2 * torch.pi*freq /113))
    fourier_basis_names.append(f'Sin {freq}')
    fourier_basis.append(torch.cos(torch.arange(113)*2 * torch.pi*freq /113))
    fourier_basis_names.append(f'Cos {freq}')
fourier_basis = torch.stack(fourier_basis, dim=0)
fourier_basis = fourier_basis/fourier_basis.norm(dim=-1)
px.imshow(fourier_basis, y=fourier_basis_names,color_continuous_scale='RdBu',).update_layout(xaxis_title="Input", yaxis_title="Cofourier_basis_namesmponent")

In [ ]:
px.imshow(fourier_basis @ fourier_basis.T)

In [ ]:
px.imshow(fourier_basis @ W_E, color_continuous_scale='RdBu', y =fourier_basis_names, title='embedding_in_fourier_basis').update_layout(xaxis_title="Residual Stream", yaxis_title="fourier_component")

In [ ]:
px.line(y=(fourier_basis @ W_E).norm(dim=-1), x=fourier_basis_names, title='embedding_in_fourier_basis').update_layout(xaxis_title="Residual Stream", yaxis_title="fourier_component")

In [ ]:
(fourier_basis @ W_E).norm(dim=-1)

In [ ]:
px.line(fourier_basis[[32, 48, 100]].mean(0))


In [ ]:
key_freq_indicies = [31, 32, 47, 48, 99, 100]
key_fourier_embed = (fourier_basis @ W_E)[key_freq_indicies]
px.imshow(key_fourier_embed@key_fourier_embed.T,color_continuous_scale='RdBu',color_continuous_midpoint=0)

# Look at frequencies in the mlp hidden activations

In [ ]:
neuron_acts_plus = cache_plus["post", 0, "mlp"][:, -1, :]
neuron_pre_acts_plus = cache_plus["pre", 0, "mlp"][:, -1, :]
neuron_acts_minus = cache_minus["post", 0, "mlp"][:, -1, :]
neuron_pre_acts_minus = cache_minus["pre", 0, "mlp"][:, -1, :]

In [ ]:
neuron_acts_plus.shape

In [ ]:
px.imshow(neuron_acts_plus[:, 1].reshape(113, 113), color_continuous_scale='RdBu', color_continuous_midpoint=0).update_layout(xaxis_title="b", yaxis_title="a")

In [ ]:
px.imshow(neuron_acts_minus[:, 1].reshape(113, 113), color_continuous_scale='RdBu', color_continuous_midpoint=0).update_layout(xaxis_title="b", yaxis_title="a")

In [ ]:
px.imshow(fourier_basis @ neuron_acts_plus[:, 1].reshape(113, 113) @ fourier_basis.T, color_continuous_scale='RdBu', color_continuous_midpoint=0, title='2d transform of neuron 1 for plus op', x=fourier_basis_names, y=fourier_basis_names).update_layout(xaxis_title="b", yaxis_title="a")

In [ ]:
px.imshow(fourier_basis @ neuron_acts_minus[:, 1].reshape(113, 113) @ fourier_basis.T, color_continuous_scale='RdBu', title='2d transform of neuron 1 for minus op', x=fourier_basis_names, y=fourier_basis_names, color_continuous_midpoint=0).update_layout(xaxis_title="b", yaxis_title="a")


### neuron clusters

In [ ]:
fourier_neuron_acts_plus = fourier_basis @ einops.rearrange(neuron_acts_plus, "(a b) neuron -> neuron a b", a=113, b=113) @ fourier_basis.T
# Center these by removing the mean - doesn't matter!
fourier_neuron_acts_plus[:, 0, 0] = 0.
print("fourier_neuron_acts", fourier_neuron_acts_plus.shape)

In [ ]:
neuron_freq_norm_plus = torch.zeros(113//2, model.cfg.d_mlp)
for freq in range(0, 113//2):
    for x in [0, 2*(freq+1) - 1, 2*(freq+1)]:
        for y in [0, 2*(freq+1) - 1, 2*(freq+1)]:
            neuron_freq_norm_plus[freq] += fourier_neuron_acts_plus[:, x, y]**2
neuron_freq_norm_plus = neuron_freq_norm_plus / fourier_neuron_acts_plus.pow(2).sum(dim=[-1, -2])[None, :]
px.imshow(neuron_freq_norm_plus, y=torch.arange(1, 113//2+1), title="Neuron Frac Explained by Freq plus", color_continuous_scale='RdBu', color_continuous_midpoint=0, aspect="auto",).update_layout(xaxis_title="Neuron", yaxis_title="Freq")

In [ ]:
px.line(neuron_freq_norm_plus.max(dim=0).values.sort().values, title="Max Neuron Frac Explained over Freqs plus").update_layout(xaxis_title="Neuron")

In [ ]:
fourier_neuron_acts_minus = fourier_basis @ einops.rearrange(neuron_acts_minus, "(a b) neuron -> neuron a b", a=113, b=113) @ fourier_basis.T
# Center these by removing the mean - doesn't matter!
fourier_neuron_acts_minus[:, 0, 0] = 0.
print("fourier_neuron_acts", fourier_neuron_acts_minus.shape)

In [ ]:
neuron_freq_norm_minus = torch.zeros(113//2, model.cfg.d_mlp)
for freq in range(0, 113//2):
    for x in [0, 2*(freq+1) - 1, 2*(freq+1)]:
        for y in [0, 2*(freq+1) - 1, 2*(freq+1)]:
            neuron_freq_norm_minus[freq] += fourier_neuron_acts_minus[:, x, y]**2
neuron_freq_norm_minus = neuron_freq_norm_minus / fourier_neuron_acts_minus.pow(2).sum(dim=[-1, -2])[None, :]
px.imshow(neuron_freq_norm_minus, y=torch.arange(1, 113//2+1), title="Neuron Frac Explained by Freq plus", color_continuous_scale='RdBu', color_continuous_midpoint=0, aspect="auto",).update_layout(xaxis_title="Neuron", yaxis_title="Freq")

In [ ]:
px.line(neuron_freq_norm_minus.max(dim=0).values.sort().values, title="Max Neuron Frac Explained over Freqs plus").update_layout(xaxis_title="Neuron")


### TODO: Maybe try to combine the above?

## Look at Unembedding

In [ ]:
W_U = model.unembed.W_U.cpu().detach()
W_O = model.blocks[0].mlp.W_out.cpu().detach()

In [ ]:
W_OU = W_O @ W_U

In [ ]:
px.imshow(W_OU, color_continuous_scale='RdBu', color_continuous_midpoint=0, aspect="auto", height=800, width=800)

In [ ]:
px.imshow(fourier_basis@W_OU.T, y=fourier_basis_names,color_continuous_scale='RdBu', color_continuous_midpoint=0, title="frequency componenets of W_OU").update_layout(xaxis_title="output", yaxis_title="Freq")

In [ ]:
px.line(y=(fourier_basis@W_OU.T).T.norm(dim=0), x=fourier_basis_names,title="normed frequency componenets of W_OU").update_layout(xaxis_title="output", yaxis_title="Freq")

In [ ]:
## show that corresponding the frequency component of W_OU output for the neuron activations with a frequency 16 is also 16 (this is for the final cos(w) output in the the trig addition)

In [ ]:
neurons_16 = neuron_freq_norm_plus[16-1] > 0.7
px.line(y=(fourier_basis@W_OU[neurons_16].T).T.norm(dim=0), x=fourier_basis_names,title="normed frequency componenets of W_OU").update_layout(xaxis_title="output", yaxis_title="Freq")


## Looking at plus and minus

In [ ]:
px.imshow(neuron_acts_plus[:, 2].reshape(113, 113), color_continuous_scale='RdBu', color_continuous_midpoint=0, title = 'plus operation neuron activations [neuron 2]').update_layout(xaxis_title="b", yaxis_title="a")

In [ ]:
px.imshow(neuron_acts_minus[:, 2].reshape(113, 113), color_continuous_scale='RdBu', color_continuous_midpoint=0, title = 'minus operation neuron activations [neuron 2]').update_layout(xaxis_title="b", yaxis_title="a")

In [ ]:
import plotly.subplots as sp
import numpy as np
max_plots = 25

num_plots = min(max_plots, neuron_acts_plus.shape[1])
grid_size = int(2)

fig = sp.make_subplots(rows=num_plots, cols=2, subplot_titles=[j for i in range(num_plots) for j in [f"Plus Neuron {i}", f"Minus Neuron {i}"]])

for i in range(num_plots):
    reshaped_data_plus = neuron_acts_plus[:, i].reshape(113, 113)
    max_val_plus = reshaped_data_plus.max()
    reshaped_data_minus = neuron_acts_minus[:, i].reshape(113, 113)
    max_val_minus = reshaped_data_minus.max()
    overall_max = max([max_val_plus, max_val_minus])
    normalized_data_plus = reshaped_data_plus / (overall_max + 1e-8)
    normalized_data_minus = reshaped_data_minus / (overall_max + 1e-8)
    heatmap = px.imshow(
        normalized_data_plus,
    )
    for trace in heatmap.data:
        fig.add_trace(trace, row=i+1, col=1)


    heatmap = px.imshow(
        normalized_data_minus,
    )
    for trace in heatmap.data:
        fig.add_trace(trace, row=i+1, col=2)

fig.update_layout(
    height=500 * max_plots, width=grid_size * 500,
    title_text="Neuron Activations",
    coloraxis=dict(cmid=0, colorscale='RdBu')
)
fig.show()



In [ ]:
plus_not_minus = []
minus_not_plus = []
for i in range(512):
    if neuron_acts_minus[:, i].sum()/neuron_acts_plus[:, i].sum() < 0.1:
        plus_not_minus.append(i)
    if neuron_acts_plus[:, i].sum()/neuron_acts_minus[:, i].sum() < 0.1:
        minus_not_plus.append(i)


all_ = plus_not_minus + minus_not_plus
num_plots = len(all_)
grid_size = int(2)

fig = sp.make_subplots(rows=num_plots, cols=2, subplot_titles=[j for i in range(num_plots) for j in [f"Plus Neuron {i}", f"Minus Neuron {i}"]])

for i, neuron_index in enumerate(all_):
    reshaped_data_plus = neuron_acts_plus[:, neuron_index].reshape(113, 113)
    max_val_plus = reshaped_data_plus.max()
    reshaped_data_minus = neuron_acts_minus[:, neuron_index].reshape(113, 113)
    max_val_minus = reshaped_data_minus.max()
    overall_max = max([max_val_plus, max_val_minus])
    normalized_data_plus = reshaped_data_plus / (overall_max + 1e-8)
    normalized_data_minus = reshaped_data_minus / (overall_max + 1e-8)
    heatmap = px.imshow(
        normalized_data_plus,
    )
    for trace in heatmap.data:
        fig.add_trace(trace, row=i+1, col=1)


    heatmap = px.imshow(
        normalized_data_minus,
    )
    for trace in heatmap.data:
        fig.add_trace(trace, row=i+1, col=2)

fig.update_layout(
    height=500 * max_plots, width=grid_size * 500,
    title_text="Neuron Activations",
    coloraxis=dict(cmid=0, colorscale='RdBu')
)
fig.show()

## ablating neurons that only fire for one operation

In [67]:
from transformer_lens import utils
def plus_mlp_neuron_ablation_hook(
    value,
    hook
):
    print(f"Shape of the value tensor: {value.shape}")
    value[:, :, plus_not_minus] = 0.
    return value

plus_neurons_ablated_ablated_plus_results = model.run_with_hooks(
    plus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        plus_mlp_neuron_ablation_hook
        )]
    )
plus_neurons_ablated_ablated_minus_results = model.run_with_hooks(
    minus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        plus_mlp_neuron_ablation_hook
        )]
    )

def minus_mlp_neuron_ablation_hook(
    value,
    hook
):
    value[:, :, minus_not_plus] = 0.
    return value

minus_neurons_ablated_ablated_plus_results = model.run_with_hooks(
    plus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        minus_mlp_neuron_ablation_hook
        )]
    )

minus_neurons_ablated_ablated_minus_results = model.run_with_hooks(
    minus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        minus_mlp_neuron_ablation_hook
        )]
    )

Shape of the value tensor: torch.Size([12769, 4, 512])
Shape of the value tensor: torch.Size([12769, 4, 512])


In [68]:
from sklearn.metrics import precision_score
plus_dataset_labels = (plus_dataset[:,0] + plus_dataset[:,2]) % 113
minus_dataset_labels = (minus_dataset[:,0] - minus_dataset[:,2]) % 113

plus_neurons_ablated_ablated_plus_predictions = plus_neurons_ablated_ablated_plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
plus_neurons_ablated_ablated_plus_precision = precision_score(plus_dataset_labels, plus_neurons_ablated_ablated_plus_predictions, average='macro')

plus_neurons_ablated_ablated_minus_predictions = plus_neurons_ablated_ablated_minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
plus_neurons_ablated_ablated_minus_precision = precision_score(minus_dataset_labels, plus_neurons_ablated_ablated_minus_predictions, average='macro')

minus_neurons_ablated_ablated_plus_predictions = minus_neurons_ablated_ablated_plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
minus_neurons_ablated_ablated_plus_precision = precision_score(plus_dataset_labels, minus_neurons_ablated_ablated_plus_predictions, average='macro')

minus_neurons_ablated_ablated_minus_predictions = minus_neurons_ablated_ablated_minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
minus_neurons_ablated_ablated_minus_precision = precision_score(minus_dataset_labels, minus_neurons_ablated_ablated_minus_predictions, average='macro')


print(f'after ablating all but neurons only active in plus: plus precision {plus_neurons_ablated_ablated_plus_precision} minus precision {plus_neurons_ablated_ablated_minus_precision}')
print(f'after ablating all but neurons only active in minus: plus precision {minus_neurons_ablated_ablated_plus_precision} minus precision {minus_neurons_ablated_ablated_minus_precision}')

after ablating all but neurons only active in plus: plus precision 0.22997348393722042 minus precision 0.9917927362577618
after ablating all but neurons only active in minus: plus precision 0.9990725177699925 minus precision 0.3939233817539317


In [69]:
def all_but_plus_mlp_neuron_ablation_hook(
    value,
    hook
):
    value[:, :, [i for i in range(512) if i not in plus_not_minus]] = 0.
    return value

all_but_plus_neurons_ablated_ablated_plus_results = model.run_with_hooks(
    plus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        all_but_plus_mlp_neuron_ablation_hook
        )]
    )
all_but_plus_neurons_ablated_ablated_minus_results = model.run_with_hooks(
    minus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        all_but_plus_mlp_neuron_ablation_hook
        )]
    )

def all_but_minus_mlp_neuron_ablation_hook(
    value,
    hook
):
    value[:, :, [i for i in range(512) if i not in minus_not_plus]] = 0.
    return value

all_but_minus_neurons_ablated_ablated_plus_results = model.run_with_hooks(
    plus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        all_but_minus_mlp_neuron_ablation_hook
        )]
    )

all_but_minus_neurons_ablated_ablated_minus_results = model.run_with_hooks(
    minus_dataset,
    fwd_hooks=[(
        utils.get_act_name("post", 0, "mlp"),
        all_but_minus_mlp_neuron_ablation_hook
        )]
    )



In [73]:
all_but_plus_neurons_ablated_ablated_plus_predictions = all_but_plus_neurons_ablated_ablated_plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
all_but_plus_neurons_ablated_ablated_plus_precision = precision_score(plus_dataset_labels, all_but_plus_neurons_ablated_ablated_plus_predictions, average='macro', zero_division=0)

all_but_plus_neurons_ablated_ablated_minus_predictions = all_but_plus_neurons_ablated_ablated_minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
all_but_plus_neurons_ablated_ablated_minus_precision = precision_score(minus_dataset_labels, all_but_plus_neurons_ablated_ablated_minus_predictions, average='macro', zero_division=0)

all_but_minus_neurons_ablated_ablated_plus_predictions = all_but_minus_neurons_ablated_ablated_plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
all_but_minus_neurons_ablated_ablated_plus_precision = precision_score(plus_dataset_labels, all_but_minus_neurons_ablated_ablated_plus_predictions, average='macro', zero_division=0)

all_but_minus_neurons_ablated_ablated_minus_predictions = all_but_minus_neurons_ablated_ablated_minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
all_but_minus_neurons_ablated_ablated_minus_precision = precision_score(minus_dataset_labels, all_but_minus_neurons_ablated_ablated_minus_predictions, average='macro', zero_division=0)


print(f'after ablating all but neurons only active in plus: plus precision {all_but_plus_neurons_ablated_ablated_plus_precision} minus precision {all_but_plus_neurons_ablated_ablated_minus_precision}')
print(f'after ablating all but neurons only active in minus: plus precision {all_but_minus_neurons_ablated_ablated_plus_precision} minus precision {all_but_minus_neurons_ablated_ablated_minus_precision}')

after ablating all but neurons only active in plus: plus precision 0.00806736360449042 minus precision 0.0005453249533054084
after ablating all but neurons only active in minus: plus precision 0.0011633032147423442 minus precision 0.014751908804845129


In [74]:
# do some recursive albation studies

In [85]:
from tqdm import tqdm
plus_results = model(plus_dataset)
plus_predictions = plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
plus_precision = precision_score(plus_dataset_labels, plus_predictions, average='macro', zero_division=0)
print(plus_precision)
plus_ablate_list = []
ablation_precisions = []
tau = 0.001 #taking inspiration from https://arxiv.org/pdf/2304.14997
current_precision_score = plus_precision
for i in tqdm(range(512)):
    new_ablate_list = plus_ablate_list.copy()
    new_ablate_list.append(i)
    def ablation_hook(
        value,
        hook
    ):
        value[:, :, new_ablate_list] = 0.
        return value

    ablated_plus_results = model.run_with_hooks(
        plus_dataset,
        fwd_hooks=[(
            utils.get_act_name("post", 0, "mlp"),
            ablation_hook
            )]
        )
    ablated_plus_predictions = ablated_plus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
    ablated_plus_precision = precision_score(plus_dataset_labels, ablated_plus_predictions, average='macro', zero_division=0)

    if current_precision_score - ablated_plus_precision < tau:
        current_precision_score = ablated_plus_precision
        plus_ablate_list = new_ablate_list.copy()
        ablation_precisions.append(ablated_plus_precision)
# print(new_ablate_list)
print([i for i in range(512) if i not in plus_ablate_list])
print(ablation_precisions)

1.0


100%|██████████| 512/512 [02:05<00:00,  4.06it/s]

[66, 70, 80, 88, 92, 102, 103, 126, 142, 147, 150, 167, 169, 172, 173, 178, 179, 180, 190, 196, 199, 202, 204, 213, 214, 222, 235, 237, 239, 246, 247, 256, 261, 265, 271, 273, 274, 277, 282, 288, 289, 290, 300, 315, 318, 319, 321, 322, 325, 329, 331, 337, 340, 341, 343, 347, 350, 351, 352, 353, 355, 356, 357, 359, 360, 361, 370, 372, 373, 376, 377, 378, 379, 384, 389, 395, 399, 402, 404, 405, 407, 410, 411, 416, 420, 421, 422, 423, 426, 429, 430, 431, 432, 434, 438, 442, 443, 446, 448, 452, 454, 455, 456, 457, 466, 468, 469, 472, 473, 479, 481, 491, 502, 503, 511]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9998460946517892, 0.9998460946517892, 0.9998460946517892, 0.9998460946517892, 0.9993080660415018, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9993013507219376, 0.9987606569328285, 0.9987606569328285, 0.99868437928218, 0.9987606569328285, 0.9987606569328285, 0.9988369345834768, 0.99883693

In [91]:
from tqdm import tqdm
minus_results = model(minus_dataset)
minus_predictions = minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
minus_precision = precision_score(minus_dataset_labels, minus_predictions, average='macro', zero_division=0)
print(minus_precision)
minus_ablate_list = []
ablation_precisions = []
tau = 0.001 #taking inspiration from https://arxiv.org/pdf/2304.14997
current_precision_score = minus_precision
for i in tqdm(range(512)):
    new_ablate_list = minus_ablate_list.copy()
    new_ablate_list.append(i)
    def ablation_hook(
        value,
        hook
    ):
        value[:, :, new_ablate_list] = 0.
        return value

    ablated_minus_results = model.run_with_hooks(
        minus_dataset,
        fwd_hooks=[(
            utils.get_act_name("post", 0, "mlp"),
            ablation_hook
            )]
        )
    ablated_minus_predictions = ablated_minus_results[:,-1,:].argmax(dim=-1).cpu().numpy()
    ablated_minus_precision = precision_score(minus_dataset_labels, ablated_minus_predictions, average='macro', zero_division=0)

    if current_precision_score - ablated_minus_precision < tau:
        current_precision_score = ablated_minus_precision
        minus_ablate_list = new_ablate_list.copy()
        ablation_precisions.append(ablated_minus_precision)
    else:
        print(ablated_minus_precision)
# print(new_ablate_list)
print([i for i in range(512) if i not in minus_ablate_list])
print(ablation_precisions)

1.0


  5%|▍         | 24/512 [00:04<01:28,  5.49it/s]

0.998525049851677


  8%|▊         | 43/512 [00:08<01:23,  5.61it/s]

0.998214539061427


...

0.9825908144270673


 99%|█████████▉| 509/512 [01:38<00:00,  5.43it/s]

0.9867719735904442


100%|██████████| 512/512 [01:39<00:00,  5.17it/s]

0.8850084699216573
0.9847029360588099
[23, 41, 67, 75, 80, 102, 106, 134, 138, 142, 147, 150, 154, 159, 163, 169, 172, 176, 178, 179, 180, 187, 188, 190, 199, 201, 208, 214, 222, 225, 239, 246, 256, 264, 265, 267, 271, 272, 273, 274, 282, 288, 308, 310, 311, 315, 319, 321, 322, 325, 329, 340, 341, 343, 350, 352, 354, 356, 357, 359, 360, 369, 370, 372, 375, 377, 381, 383, 384, 386, 389, 395, 399, 402, 405, 407, 409, 410, 411, 415, 416, 418, 420, 422, 423, 426, 429, 430, 431, 432, 434, 437, 438, 440, 441, 442, 446, 452, 455, 456, 457, 458, 463, 466, 467, 468, 469, 472, 473, 479, 481, 482, 491, 495, 498, 500, 502, 507, 510, 511]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.999844744604875, 0.999844744604875, 0.9999223723024375, 0.9999223723024375, 0.9999223723024375, 0.9999223723024375, 0.999922372302437

# Staged Training

In [ ]:
results = torch.load('output_plus_and_minus_staged.pt')
plt.plot(results['test_losses'])
plt.plot(results['train_losses'])
plt.yscale('log')



In [ ]:
for operation in results['operations_losses']:
    plt.plot(operation['test_losses'])
    plt.plot(operation['train_losses'])
plt.yscale('log')